In [1]:
import os
import numpy as np
import pandas as pd
import random
import string
random.seed(123)
import datetime as dt

# import warnings
# warnings.filterwarnings('ignore','RuntimeWarning')

import nltk
import re
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.collocations import BigramCollocationFinder, TrigramCollocationFinder

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
f1 = open('EMR text files/Adam Pie.txt','r',encoding='latin').readlines()
f2 = open('EMR text files/Bill Windows.txt','r',encoding='latin').readlines()
f3 = open('EMR text files/Billy Gato.txt','r',encoding='latin').readlines()
f4 = open('EMR text files/Cherie Amore.txt','r',encoding='latin').readlines()
f5 = open('EMR text files/John Donut.txt','r',encoding='latin').readlines()
f6 = open('EMR text files/Monica Latte.txt','r',encoding='latin').readlines()
f7 = open('EMR text files/Steve Apple.txt','r',encoding='latin').readlines()
f8 = open('EMR text files/Tom Gellato.txt','r',encoding='latin').readlines()
f9 = open('EMR text files/Wendy See.txt','r',encoding='latin').readlines()

names=['Adam Pie','Bill Windows','Billy Gato','Cherie Amore','John Donut','Monica Latte','Steve Apple','Tom Gellato','Wendy See']

Document = [f1,f2,f3,f4,f5,f6,f7,f8,f9]

In [9]:
data_frame = pd.DataFrame({'names':names,'document':Document})

In [10]:
data_frame

,names,document
0,Adam Pie,"[Use for January 2011 abstraction\n, \n, WeSer..."
1,Bill Windows,"[Use for April 2011 abstraction \n, \n, WeServ..."
2,Billy Gato,"[Use for October 2010 abstraction\n, \n, WeSer..."
3,Cherie Amore,"[Use for October 2010 abstraction\n, \n, WeSer..."
4,John Donut,"[WeServeEveryone Clinic\t \n, 1111 First Stree..."
5,Monica Latte,"[WeServeEveryone Clinic\t \n, 1111 First Stree..."
6,Steve Apple,"[Use for April 2011 abstraction\n, \n, WeServe..."
7,Tom Gellato,"[Use for January 2011 abstraction\n, \n, WeSer..."
8,Wendy See,"[Use for October 2010 abstraction \n, \n, WeSe..."


In [40]:
# Function to Preprocess the Reviews
def clean_doc(doc):
    
    # Tokenization
    tokens = doc.split(' ')
    
    # Converting into lower case
    tokens = [w.lower() for w in tokens]
    
    # Punctuation removal
    tokens = [re.sub(r"[^a-zA-Z#\s]",'',i) for i in tokens]
    tokens = [re.sub(r"[\r\n\t]",'',i) for i in tokens]
    
    # Stopwords removal
    stop_words = set(stopwords.words('english'))
    tokens = [w for w in tokens if not w in stop_words]
    
    # Lemmatization
    lmtzr = nltk.stem.WordNetLemmatizer()
    tokens = [lmtzr.lemmatize(w) for w in tokens]
    
    # Stemming
    stemmer = nltk.stem.SnowballStemmer('english')
    tokens = [stemmer.stem(w) for w in tokens]
    
    # filter out short tokens
    tokens = [word for word in tokens if len(word) > 1]
    return tokens

In [41]:
data_frame['modified_document'] = data_frame['document'].apply(lambda x: [clean_doc(each_line) for each_line in x] )

In [42]:
data_frame['modified_document']

0    [[use, januari, abstract], [], [weserveeveryon...
1    [[use, april, abstract], [], [weserveeveryon, ...
2    [[use, octob, abstract], [], [weserveeveryon, ...
3    [[use, octob, abstract], [], [weserveeveryon, ...
4    [[weserveeveryon, clinic], [first, street, cal...
5    [[weserveeveryon, clinic], [first, street, cal...
6    [[use, april, abstract], [], [weserveeveryon, ...
7    [[use, januari, abstract], [], [weserveeveryon...
8    [[use, octob, abstract], [], [weserveeveryon, ...
Name: modified_document, dtype: object

In [43]:
data_frame['modified_document'] = data_frame['modified_document'].apply(lambda x: [x1 for xs in x for x1 in xs])

In [44]:
data_frame

,names,document,modified_document,Document_except_rare_freq
0,Adam Pie,"[Use for January 2011 abstraction\n, \n, WeSer...","[use, januari, abstract, weserveeveryon, clini...","[soc, hiv, reason, temp, club, sub, advers, ro..."
1,Bill Windows,"[Use for April 2011 abstraction \n, \n, WeServ...","[use, april, abstract, weserveeveryon, clinic,...","[soc, hiv, reason, temp, sub, advers, routin, ..."
2,Billy Gato,"[Use for October 2010 abstraction\n, \n, WeSer...","[use, octob, abstract, weserveeveryon, clinic,...","[soc, hiv, reason, temp, club, sub, advers, ro..."
3,Cherie Amore,"[Use for October 2010 abstraction\n, \n, WeSer...","[use, octob, abstract, weserveeveryon, clinic,...","[soc, hiv, reason, temp, sub, advers, routin, ..."
4,John Donut,"[WeServeEveryone Clinic\t \n, 1111 First Stree...","[weserveeveryon, clinic, first, street, califo...","[soc, hiv, reason, temp, club, sub, advers, ro..."
5,Monica Latte,"[WeServeEveryone Clinic\t \n, 1111 First Stree...","[weserveeveryon, clinic, first, street, califo...","[soc, hiv, reason, temp, club, sub, advers, ro..."
6,Steve Apple,"[Use for April 2011 abstraction\n, \n, WeServe...","[use, april, abstract, weserveeveryon, clinic,...","[soc, hiv, reason, temp, club, sub, advers, ro..."
7,Tom Gellato,"[Use for January 2011 abstraction\n, \n, WeSer...","[use, januari, abstract, weserveeveryon, clini...","[soc, hiv, reason, temp, club, sub, advers, ro..."
8,Wendy See,"[Use for October 2010 abstraction \n, \n, WeSe...","[use, octob, abstract, weserveeveryon, clinic,...","[soc, hiv, reason, temp, club, sub, advers, ro..."


In [45]:
# Function to convert nested lists to single list
def unlist_words(a):
    temp = [x for xs in a for x in xs]
    return temp

In [46]:
corpus = unlist_words(data_frame['modified_document'])

In [47]:
len(corpus)

5426

In [80]:
a = nltk.FreqDist(corpus)
d = pd.DataFrame({'Word': list(a.keys()),
                  'Count': list(a.values())})
frequent_words = d.nlargest(columns="Count", n = 350)
rare_words = d.nsmallest(columns="Count", n = 100) 

In [81]:
data_frame['Document_except_rare_freq'] = data_frame['modified_document'].apply(lambda x: list((set(x)-set(rare_words.Word))-set(frequent_words.Word)))

In [82]:
data_frame

,names,document,modified_document,Document_except_rare_freq
0,Adam Pie,"[Use for January 2011 abstraction\n, \n, WeSer...","[use, januari, abstract, weserveeveryon, clini...","[neg, bhi, microalb, urn, race, lab, social, m..."
1,Bill Windows,"[Use for April 2011 abstraction \n, \n, WeServ...","[use, april, abstract, weserveeveryon, clinic,...","[flowsheet, night, hypoglycem, account, bill, ..."
2,Billy Gato,"[Use for October 2010 abstraction\n, \n, WeSer...","[use, octob, abstract, weserveeveryon, clinic,...","[neg, lab, race, social, monitor, report, acco..."
3,Cherie Amore,"[Use for October 2010 abstraction\n, \n, WeSer...","[use, octob, abstract, weserveeveryon, clinic,...","[social, account, millennium, glucos, consult,..."
4,John Donut,"[WeServeEveryone Clinic\t \n, 1111 First Stree...","[weserveeveryon, clinic, first, street, califo...","[neg, bhi, microalb, urn, lab, race, monitor, ..."
5,Monica Latte,"[WeServeEveryone Clinic\t \n, 1111 First Stree...","[weserveeveryon, clinic, first, street, califo...","[lisinopril, auscult, report, glucos, microalb..."
6,Steve Apple,"[Use for April 2011 abstraction\n, \n, WeServe...","[use, april, abstract, weserveeveryon, clinic,...","[flowsheet, lisinopril, marri, neg, steve, per..."
7,Tom Gellato,"[Use for January 2011 abstraction\n, \n, WeSer...","[use, januari, abstract, weserveeveryon, clini...","[neg, tom, microalb, urn, race, social, monito..."
8,Wendy See,"[Use for October 2010 abstraction \n, \n, WeSe...","[use, octob, abstract, weserveeveryon, clinic,...","[social, cap, report, millennium, health, lowe..."


### Ngrams

In [83]:
# ngrams
def ngrams(CollocationFinder,x):
    a = CollocationFinder.from_words(x).ngram_fd
    d = pd.DataFrame({'ngrams': list(a.keys()),
                  'Count': list(a.values())})
    # selecting top 25 bigrams     
    ngrams = d.nlargest(columns="Count", n = 25)
    return ngrams.ngrams

In [84]:
data_frame['bigrams'] = data_frame['Document_except_rare_freq'].apply(lambda x: list(ngrams(BigramCollocationFinder,x)))

In [85]:
data_frame['trigrams'] = data_frame['Document_except_rare_freq'].apply(lambda x: list(ngrams(TrigramCollocationFinder,x)))

In [86]:
data_frame

,names,document,modified_document,Document_except_rare_freq,bigrams,trigrams
0,Adam Pie,"[Use for January 2011 abstraction\n, \n, WeSer...","[use, januari, abstract, weserveeveryon, clini...","[neg, bhi, microalb, urn, race, lab, social, m...","[(neg, bhi), (bhi, microalb), (microalb, urn),...","[(neg, bhi, microalb), (bhi, microalb, urn), (..."
1,Bill Windows,"[Use for April 2011 abstraction \n, \n, WeServ...","[use, april, abstract, weserveeveryon, clinic,...","[flowsheet, night, hypoglycem, account, bill, ...","[(flowsheet, night), (night, hypoglycem), (hyp...","[(flowsheet, night, hypoglycem), (night, hypog..."
2,Billy Gato,"[Use for October 2010 abstraction\n, \n, WeSer...","[use, octob, abstract, weserveeveryon, clinic,...","[neg, lab, race, social, monitor, report, acco...","[(neg, lab), (ago, year), (ophthalmolog, lower...","[(neg, lab, race), (lab, race, social), (percu..."
3,Cherie Amore,"[Use for October 2010 abstraction\n, \n, WeSer...","[use, octob, abstract, weserveeveryon, clinic,...","[social, account, millennium, glucos, consult,...","[(social, account), (account, millennium), (mi...","[(social, account, millennium), (account, mill..."
4,John Donut,"[WeServeEveryone Clinic\t \n, 1111 First Stree...","[weserveeveryon, clinic, first, street, califo...","[neg, bhi, microalb, urn, lab, race, monitor, ...","[(neg, bhi), (lisinopril, better), (ophthalmol...","[(neg, bhi, microalb), (bhi, microalb, urn), (..."
5,Monica Latte,"[WeServeEveryone Clinic\t \n, 1111 First Stree...","[weserveeveryon, clinic, first, street, califo...","[lisinopril, auscult, report, glucos, microalb...","[(lisinopril, auscult), (auscult, report), (re...","[(lisinopril, auscult, report), (auscult, repo..."
6,Steve Apple,"[Use for April 2011 abstraction\n, \n, WeServe...","[use, april, abstract, weserveeveryon, clinic,...","[flowsheet, lisinopril, marri, neg, steve, per...","[(flowsheet, lisinopril), (lisinopril, marri),...","[(flowsheet, lisinopril, marri), (lisinopril, ..."
7,Tom Gellato,"[Use for January 2011 abstraction\n, \n, WeSer...","[use, januari, abstract, weserveeveryon, clini...","[neg, tom, microalb, urn, race, social, monito...","[(neg, tom), (night, ago), (white, lower), (op...","[(neg, tom, microalb), (tom, microalb, urn), (..."
8,Wendy See,"[Use for October 2010 abstraction \n, \n, WeSe...","[use, octob, abstract, weserveeveryon, clinic,...","[social, cap, report, millennium, health, lowe...","[(social, cap), (cap, report), (report, millen...","[(social, cap, report), (cap, report, millenni..."


### Term Frequency

In [87]:
nltk.FreqDist(data_frame['Document_except_rare_freq'][0])

FreqDist({'neg': 1, 'bhi': 1, 'microalb': 1, 'urn': 1, 'race': 1, 'lab': 1, 'social': 1, 'monitor': 1, 'report': 1, 'account': 1, ...})

In [88]:
data_frame['term_frequency'] = data_frame['Document_except_rare_freq'].apply(lambda x: nltk.FreqDist(x))

In [90]:
data_frame.term_frequency[0]

FreqDist({'neg': 1, 'bhi': 1, 'microalb': 1, 'urn': 1, 'race': 1, 'lab': 1, 'social': 1, 'monitor': 1, 'report': 1, 'account': 1, ...})

In [91]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [92]:
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1,2))

In [93]:
tfidf_vectorizer.fit(corpus)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

### Inverse Document Frequency

In [94]:
idf = tfidf_vectorizer.idf_

In [95]:
idf

array([7.29655668, 7.51970023, 5.83794166, 7.65323163, 7.4019172 ,
       7.29655668, 7.29655668, 7.65323163, 7.29655668, 7.29655668,
       7.29655668, 7.29655668, 7.8073823 , 8.50052949, 7.4019172 ,
       7.29655668, 7.29655668, 7.29655668, 8.90599459, 8.90599459,
       7.29655668, 8.90599459, 7.29655668, 7.29655668, 7.29655668,
       7.98970386, 8.90599459, 8.50052949, 8.90599459, 7.51970023,
       7.4019172 , 7.65323163, 8.21284741, 8.90599459, 7.29655668,
       7.29655668, 8.50052949, 7.98970386, 7.8073823 , 6.70877002,
       7.65323163, 7.4019172 , 7.8073823 , 7.8073823 , 7.29655668,
       8.50052949, 7.29655668, 6.82655305, 6.65470279, 8.90599459,
       7.29655668, 5.86147216, 6.16515457, 7.29655668, 7.29655668,
       7.51970023, 7.4019172 , 7.29655668, 7.4019172 , 5.83794166,
       8.21284741, 7.4019172 , 7.65323163, 6.65470279, 6.42108794,
       5.86147216, 6.26693726, 5.77050038, 8.50052949, 7.29655668,
       7.29655668, 7.29655668, 7.4019172 , 8.90599459, 5.93558

### TFIDF

In [96]:
data_frame['tfidf'] = data_frame['Document_except_rare_freq'].apply(lambda x: tfidf_vectorizer.transform(x).toarray())

In [97]:
data_frame

,names,document,modified_document,Document_except_rare_freq,bigrams,trigrams,term_frequency,tfidf
0,Adam Pie,"[Use for January 2011 abstraction\n, \n, WeSer...","[use, januari, abstract, weserveeveryon, clini...","[neg, bhi, microalb, urn, race, lab, social, m...","[(neg, bhi), (bhi, microalb), (microalb, urn),...","[(neg, bhi, microalb), (bhi, microalb, urn), (...","{'neg': 1, 'bhi': 1, 'microalb': 1, 'urn': 1, ...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
1,Bill Windows,"[Use for April 2011 abstraction \n, \n, WeServ...","[use, april, abstract, weserveeveryon, clinic,...","[flowsheet, night, hypoglycem, account, bill, ...","[(flowsheet, night), (night, hypoglycem), (hyp...","[(flowsheet, night, hypoglycem), (night, hypog...","{'flowsheet': 1, 'night': 1, 'hypoglycem': 1, ...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
2,Billy Gato,"[Use for October 2010 abstraction\n, \n, WeSer...","[use, octob, abstract, weserveeveryon, clinic,...","[neg, lab, race, social, monitor, report, acco...","[(neg, lab), (ago, year), (ophthalmolog, lower...","[(neg, lab, race), (lab, race, social), (percu...","{'neg': 1, 'lab': 1, 'race': 1, 'social': 1, '...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
3,Cherie Amore,"[Use for October 2010 abstraction\n, \n, WeSer...","[use, octob, abstract, weserveeveryon, clinic,...","[social, account, millennium, glucos, consult,...","[(social, account), (account, millennium), (mi...","[(social, account, millennium), (account, mill...","{'social': 1, 'account': 1, 'millennium': 1, '...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
4,John Donut,"[WeServeEveryone Clinic\t \n, 1111 First Stree...","[weserveeveryon, clinic, first, street, califo...","[neg, bhi, microalb, urn, lab, race, monitor, ...","[(neg, bhi), (lisinopril, better), (ophthalmol...","[(neg, bhi, microalb), (bhi, microalb, urn), (...","{'neg': 1, 'bhi': 1, 'microalb': 1, 'urn': 1, ...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
5,Monica Latte,"[WeServeEveryone Clinic\t \n, 1111 First Stree...","[weserveeveryon, clinic, first, street, califo...","[lisinopril, auscult, report, glucos, microalb...","[(lisinopril, auscult), (auscult, report), (re...","[(lisinopril, auscult, report), (auscult, repo...","{'lisinopril': 1, 'auscult': 1, 'report': 1, '...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
6,Steve Apple,"[Use for April 2011 abstraction\n, \n, WeServe...","[use, april, abstract, weserveeveryon, clinic,...","[flowsheet, lisinopril, marri, neg, steve, per...","[(flowsheet, lisinopril), (lisinopril, marri),...","[(flowsheet, lisinopril, marri), (lisinopril, ...","{'flowsheet': 1, 'lisinopril': 1, 'marri': 1, ...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
7,Tom Gellato,"[Use for January 2011 abstraction\n, \n, WeSer...","[use, januari, abstract, weserveeveryon, clini...","[neg, tom, microalb, urn, race, social, monito...","[(neg, tom), (night, ago), (white, lower), (op...","[(neg, tom, microalb), (tom, microalb, urn), (...","{'neg': 1, 'tom': 1, 'microalb': 1, 'urn': 1, ...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
8,Wendy See,"[Use for October 2010 abstraction \n, \n, WeSe...","[use, octob, abstract, weserveeveryon, clinic,...","[social, cap, report, millennium, health, lowe...","[(social, cap), (cap, report), (report, millen...","[(social, cap, report), (cap, report, millenni...","{'social': 1, 'cap': 1, 'report': 1, 'millenni...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."


In [99]:
from autocorrect import Speller
spellchk = Speller(threshold=0)

In [111]:
from spellchecker import SpellChecker

ModuleNotFoundError: No module named 'spellchecker'

In [112]:
!pip install spellchecker

  Running setup.py bdist_wheel for spellchecker: started
  Running setup.py bdist_wheel for spellchecker: finished with status 'done'
  Stored in directory: C:\Users\lokesh\AppData\Local\pip\Cache\wheels\a2\e6\ad\28ab959cba7100f5c562a3d1711cd34b630734d241c4e1bd40
  Running setup.py bdist_wheel for inexactsearch: started
  Running setup.py bdist_wheel for inexactsearch: finished with status 'done'
  Stored in directory: C:\Users\lokesh\AppData\Local\pip\Cache\wheels\58\e0\c1\e3fed0e9fd1a3708bc91870fb0ba30ef88527540006763674b
  Running setup.py bdist_wheel for soundex: started
  Running setup.py bdist_wheel for soundex: finished with status 'done'
  Stored in directory: C:\Users\lokesh\AppData\Local\pip\Cache\wheels\b5\bb\e6\9a4b6be56c40aa707509bddaf6d414187461ded9db7a25a41a
  Running setup.py bdist_wheel for silpa-common: started
  Running setup.py bdist_wheel for silpa-common: finished with status 'done'
  Stored in directory: C:\Users\lokesh\AppData\Local\pip\Cache\wheels\16\4f\ba\604

thinc 6.10.3 requires msgpack<1.0.0,>=0.5.6, which is not installed.
msgpack-numpy 0.4.4.3 requires msgpack>=0.5.2, which is not installed.
distributed 1.21.8 requires msgpack, which is not installed.
tensorflow 1.10.0 has requirement numpy<=1.14.5,>=1.13.3, but you'll have numpy 1.17.0 which is incompatible.
tensorflow 1.10.0 has requirement setuptools<=39.1.0, but you'll have setuptools 41.0.1 which is incompatible.
tensorflow 1.10.0 has requirement tensorboard<1.11.0,>=1.10.0, but you'll have tensorboard 1.14.0 which is incompatible.
spacy 2.0.13 has requirement msgpack-numpy<0.29,<0.4.4.0murmurhash>=0.28, but you'll have msgpack-numpy 0.4.4.3 which is incompatible.
spacy 2.0.13 has requirement regex==2018.01.10, but you'll have regex 2017.11.9 which is incompatible.
cufflinks 0.16 has requirement plotly<4.0.0a0,>=3.0.0, but you'll have plotly 4.1.0 which is incompatible.
You are using pip version 10.0.1, however version 19.2.3 is available.
You should consider upgrading via the 'py

In [109]:
spellchk('tigrr')

'tiger'

In [100]:
data_frame['spell_corrected_doc'] = data_frame['modified_document'].apply(lambda x: [spellchk(word) for word in x])

In [101]:
len(data_frame['spell_corrected_doc'][0])

717

In [102]:
temp1  = data_frame['spell_corrected_doc'][0]

In [103]:
temp2  = data_frame['modified_document'][0]

In [104]:
temp1 == temp2

False

In [106]:
corrected_df = pd.DataFrame({'wrong':temp2,'right':temp1})

In [107]:
corrected_df[corrected_df['wrong'] != corrected_df['right']]

,wrong,right
1,januari,january
10,summari,summary
17,commerci,commerce
18,xxxxx,xxxix
23,piehom,pigdom
39,activ,active
41,malemarit,malemuit
50,languag,language
58,byemp,hemp
60,fulltim,fulltime


In [192]:
strtemp = ' '.join(f1)

In [206]:
re.findall('Patient ID: .+\\t|Patient ID: .+\\n',strtemp)

['Patient ID: 0000-88888\t']

In [207]:
re.findall('Birth Date: .+\\t|Birth Date: .+\\n',strtemp)

['Birth Date: 08/08/1948\t']

In [209]:
re.findall('Gender: .+\\t|Gender: .+\\n',strtemp)

['Gender: Male\t']

In [210]:
re.findall('Marital Status: .+\\t|Marital Status: .+\\n',strtemp)

['Marital Status: Married\n']

In [211]:
re.findall('Race: .+\\t|Race: .+\\n',strtemp)

['Race: White\n']

In [212]:
re.findall('Language: .+\\t|Language: .+\\n',strtemp)

['Language: English\n']

In [213]:
re.findall('Resp Prov: .+\\t|Resp Prov: .+\\n',strtemp)

['Resp Prov: Carl Savem\t']

In [214]:
re.findall('MRN: .+\\t|MRN: .+\\n',strtemp)

['MRN: MR-111-1111\n']

In [215]:
re.findall('Emp. Status: .+\\t|Emp. Status: .+\\n',strtemp)

['Emp. Status: Full-time\n']

In [216]:
re.findall('External ID: .+\\t|External ID: .+\\n',strtemp)

['External ID: MR-111-1111\n']

In [236]:
re.findall('Problems\\n\s.+',strtemp,re.MULTILINE)

['Problems\n DIABETES MELLITUS (ICD-250.)']

In [218]:
strtemp

'Use for January 2011 abstraction\n \n WeServeEveryone Clinic\t \n 1111 First Street California\n 111-111-11111   Fax: 111-111-1111\tChart Summary\n Adam Pie\n \n Home: 888-888-8888\n Male   DOB: 08/08/1948\t0000-88888\tIns: Commercial xxxxx\n                                                      \n \n Patient Information\n Name: Adam Pie\tHome Phone: 888-888-8888\n Address: 1111 Donut Road\n                 Fast Food, California  \tOffice Phone:\n Patient ID: 0000-88888\tFax:\n Birth Date: 08/08/1948\tStatus: Active\n Gender: Male\tMarital Status: Married\n Contact By: Phone \tRace: White\n Soc Sec No: 111-11-1111\tLanguage: English\n Resp Prov: Carl Savem\tMRN: MR-111-1111\n Referred by:\tEmp. Status: Full-time\n Email:\tSens Chart: No\n Home LOC: WeServeEveryone \tExternal ID: MR-111-1111\n Problems\n DIABETES MELLITUS (ICD-250.)\n HYPERTENSION, BENIGN ESSENTIAL (ICD-401.1)\n DEPRESSION (ICD-311)\n RETINOPATHY, DIABETIC (ICD-362.0)\n POLYNEUROPATHY IN DIABETES (ICD-357.2)\n \n Medica